In [20]:
# import required modules
import pandas as pd
import os

# get renumbering key
dfRenumber = pd.read_csv(r"_node_key_v832_to_v9_bh.csv")
dfRenumber

In [40]:
# select specific folder year
linYear = "Lin_2040_Needs"

# assign directories
directory = os.getcwd()
v9Directory = os.path.normpath(os.getcwd() + os.sep + os.pardir + "\Transit-v9\\" + linYear)
v832Directory = os.path.normpath(os.getcwd() + os.sep + os.pardir + "\Transit-v832\\" + linYear)

c:\Users\cday\Documents\projects\TDM-Transit-Node-Renumbering-v832-to-v9\Transit-v9\Lin_2040_Needs


In [ ]:
# iterate over files in v832 directory of selected year
for filename in os.listdir(v832Directory):
  f = os.path.join(v832Directory, filename)
  # checking if it is a file
  if os.path.isfile(f) & f.endswith('.lin'):
    print("")
    print("FILE: " + f)

    strFilename = os.path.join(v9Directory + '\\' + os.path.basename(f));
    with open(strFilename, 'w') as fo:
      fo.write(";;<<PT>><<LINE>>;;") # THIS LINE IS NEEDED FOR CUBE TO RECOGNIZE THIS AS A PT FILE
      fo.write('\n')

    # Using readlines()
    file1 = open(f, 'r')
    Lines = file1.readlines()
    
    last = Lines[-1]

    ctLINE = 0
    strNodesNewList = ""

    count = 0
    # Strips the newline character
    for line in Lines:
        count += 1
        if ctLINE==0: # write out all lines before first line that has ";"
          if line[0:1]==';' and line[0:2]!=';L' and count>1: #SKIP FIRST LINE SINCE MANUALLY CREATED PREVIOUSLY
            print (line)
            with open(strFilename, 'a') as fo:
              fo.write(line)

        #print("Line{}: {}".format(count, line.strip()))
        # find where N=
        posL = line.find("LINE NAME=")
        if posL>=0 or line is last: # LINE found since code only processes when another line is reached, so manually have to read last line
          if line is last:
            LINEstring = LINEstring + line.replace(";", "").strip()
          if ctLINE: # previously had been found, so this means that a new line is found
            newLINE = ""
            #print(LINEstring)

            # add code to change nodes

            # find where N=
            curPos = 0
            
            while curPos<len(LINEstring):
              posN = LINEstring.find("N=",curPos) # find the next N marker
              #print (strNodesNewList)
              

              if posN>=0:
                
                newLINE = newLINE + "," + LINEstring[curPos:posN] + "N="
                
                strNodesNewList = ""
                #print ("N Found at " + str(posN))
                curPos = posN + len("N=")

                while curPos<len(LINEstring): # if N foound then loop through rest of string until non-number found
                  # find next comma
                  posComma= LINEstring.find(",",curPos)
                  if posComma>0:
                    posNext = posComma
                  else:
                    posNext = len(LINEstring)
                
                  #print (posNext)
                  _sub = LINEstring[curPos:posNext].strip()
                  if _sub[0:1]=='-':
                    bNeg = True;
                  else:
                    bNeg = False;
                  curPos = posNext + 1
                  if _sub.strip('-').isnumeric(): # strip negative sign for isnumeric valuation, if it is numeric than this is a node number
                    iNode = _sub.strip('-')
                    
                    dfNodeNew = dfRenumber.loc[dfRenumber.N_v832_correct == int(iNode), 'N_V9']

                    if dfNodeNew.size>0:
                      iNodeNew = dfNodeNew.iloc[0]
                    else:
                      iNodeNew = 999999

                    if bNeg:
                      iNodeNew = iNodeNew * -1

                    strNodeNew = str(iNodeNew)
                    if len(strNodesNewList):
                      strNodesNewList = strNodesNewList + "," + strNodeNew
                    else:
                      strNodesNewList = strNodeNew
                    #print ('N v832: ' + _sub + ' N v9: ' + strNodeNew)
                  else: # if not a number than break out so that it can find the next N marker
                    #print ('OTHER:' + _sub)
                    newLINE = newLINE + strNodesNewList + "," + _sub
                    break
            #print (strNodesNewList)
            newLINE = newLINE + strNodesNewList
            newLINE = newLINE.strip(',')
            print(newLINE)
            with open(strFilename, 'a') as fo:
              fo.write(newLINE)
              fo.write('\n')

          #print (count)
          ctLINE = ctLINE + 1
          LINEstring = line.strip()
        elif (ctLINE): # continue appending LINE data if one has previously been found
          LINEstring = LINEstring + line.replace(";", "").strip()